## 爬虫基础

1. 前端：html（骨架标签）、 css（美化）、 js（加密、校验）

2. 后端：python、 php、 java、 go

3. JS逆向（node.js）

4. 浏览器自动化调试Selenium


5. Tips : VScode 替换选中字符加上引号
    1. ctrl + H
    2. 选择 .* 正则
    3. (.*?):(.*)  to  '$1':'$2'

### 1. 静态网页 通过-内容可看到
 在源代码中直接可以搜索的到页面内的链接关键字
   1. requests库获取网页对象，`.text`文本属性、`.contents`二进制属性、`.json()`把json转换成字典
      > python中用print打印text可观察出，双引号是json格式、单引号是字典格式；
      > 也可以用pandas read_json,to_excel 直接转化成excel文件
   2. 爬虫校验： # 一定是本地校验
      1. 地址  协议： ip网址 / 路径 ？参数（+猫+路由）
      2. XHR断点调试法：我们要使用路径，在源代码的右边找到XHR/提取断点，+一个路径进去，然后刷新页面，在作用域中调试的内容的输出，单步调试直到发现带有该路径的信息出现。
      3. requestHeaders：Headers校验的头内容
      4. options：写明该路径下接口真正的请求方式和参数内容

### 2. 动态网页（AJAX渲染） 通过-网络（抓包工具抓取）
刷新观察加载内容的速度差距 or 在源代码中直接搜索不到页面内的链接关键字，提示是使用XHR加载

> 数据一般通过Json文件传递，通过预览查看是否是我们想要的信息



#### 2.1 post表单中sign签字加密
批量采集需要控制翻页，翻页就是在post的data参数中（有可能配合着请求头的u-sign加密翻页使用）
1. 网络中抓包，得到负载的表单，观察有加密数据
   1. 载荷数据需要post JSON数据
   2. 表单数据post 字典数据
2. CTRL+SHIFT+F 全局查找 sign或（js赋值方式 sign= ; sign:），依次打上断点
3. 刷新，输入你好。调试观察变量。（调试完成后记得右键清除断点）

> ~~在线加密解密网站，瞎猫碰死耗子~~


```python
import time
import random
from hashlib import md5
timenow = str(int(time.time()*100))
sign = timenow+str(random(0,9)) + 'asknjviwnef13nakv/*65f4'
# sign的格式是字符串，需要编码成二进制再放进md5函数，然后再转为16进制输出
md5(sign.encode()).hexdigets()
```

#### 2.2 动态地进行断点调试+JS代码补环境

1. 动态网页 全局查找 返回太多无法查证时。
    > 但是一定会发送信息，调用网址路径的接口

    1. 复制**ip网址和路径**，在源代码右侧中添加XHR/提取断点
    2. 然后刷新，在作用域下方找到**调用堆栈**:从上到下依次调用了什么函数或变量
    3. 定位到参数加密位置。 要有耐心

2. 扣JS代码
   1. 首先观察函数做了什么，**先看返回值**怎么组成；
   2. 从Python这个胶水语言编译JS代码执行
        1. 浏览器使用JavaScript
        2. python使用Node.js # 需下载安装 https://blog.csdn.net/maotaolaodie/article/details/126596042
        3. 新建.js文件，扣代码，补环境 // console.log(j()) # 注释，print
            ```python
            import execjs
            with open('xin.js','r')as f:
                js_code = f.read()
            js_ex = execjs.compile(js_code)
            nonce = js_ex.call('j) 
            print(nonce)
            ```

#### 2.3 服务端返回加密的响应数据如何本地解析
案例：百度指数
1. **页面太多而且信息加密了**，**清除后重新抓包**选择一个按钮进行加载。
2. 客户端一定会从服务端返回的响应数据中取值，可以**全局搜索key值** 
   > 例如Json文件的选取 d['data']['indexnorm'][1]['users']
3. 两条路
   1. 扣js #$$ 在js中代表 要么True执行前面的要么执行后面的；
   2. 看懂函数，手写python函数